In [ ]:
# Code: https://github.com/deepakat002/numberplaterecognition
# Model(best.pt): https://github.com/pracool/ANPR-with-YoloV5

In [11]:
# %pip install easyocr==1.6.2
# %conda install pytesseract=0.3.10 -y

Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/Caskroom/miniforge/base/envs/park-3.9

  added / updated specs:
    - pytesseract=0.3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    leptonica-1.82.0           |       h37c441e_0         1.6 MB
    libarchive-3.6.1           |       he3a3bf9_0         751 KB
    openjpeg-2.3.0             |       h7a6adac_2         274 KB
    pillow-9.2.0               |   py39h4d1bdd5_1         603 KB
    pytesseract-0.3.10         |   py39hca03da5_0          39 KB
    tesseract-5.2.0            |       hc377ac9_0       167.9 MB
    ------------------------------------------------------------
                                           Total:       171.2 MB

The following NEW packages will be INSTALLED:

  bzip2              pkgs/main/osx-arm64::bzip2-1.0.8-h620ffc9_4
  freetype           pkgs/main/osx-arm64::freet

In [1]:
import easyocr

/opt/homebrew/Caskroom/miniforge/base/envs/park-3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# importing required libraries
import cv2
import numpy as np
import pytesseract
import torch
from datetime import datetime
import csv
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
# DEFINING GLOBAL VARIABLE
EASY_OCR = easyocr.Reader(['en'])  # initiating easyocr
OCR_TH = 0.2

# Global Variables for text regecnitioin
SCREEN_WIDTH = 128
SCREEN_HEIGHT = 64


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
# -------------------------------------- function to run detection ---------------------------------------------------------
def detectx(frame, model):
    frame = [frame]
    results = model(frame)
    labels, cordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
    return labels, cordinates

In [80]:

# ---------------------------- function to recognize license plate --------------------------------------


# function to recognize license plate numbers using Tesseract OCR
def recognize_plate_easyocr(img, coords, reader, region_threshold):
    # separate coordinates from box
    xmin, ymin, xmax, ymax = coords
    # get the subimage that makes up the bounded region and take an additional 5 pixels on each side
    #nplate = img[int(ymin)+15:int(ymax)-5, int(xmin)+12:int(xmax)-12]
    # cropping the number plate from the whole image

    nplate = img[int(ymin):int(ymax), int(xmin):int(xmax)]
    # delete the left side with 10 px of the image
    nplate = nplate[:, 35:]

    # convert nplate to gray
    nplate = cv2.cvtColor(nplate, cv2.COLOR_BGR2GRAY)
    #display(Image.fromarray(nplate))
    
    scale_factor = 6
    scaled_img = cv2.resize(nplate[1:50, 0:SCREEN_WIDTH], (0,0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
    assert(scaled_img is not None)
    #display(Image.fromarray(scaled_img))

    thres,thres_img = cv2.threshold(nplate, 0, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU)
    assert(thres_img is not None)
    #display(Image.fromarray(thres_img))

    ############################ Using Pytesseract to extract text from image ########################################
    text_1 = pytesseract.image_to_string(thres_img, config='--user-words words.txt config.txt')
    print('Text prediciton using Pytesseract:"{}"'.format(text_1)) 
    #############################################################################################################

    ####################################### using EasyORC to extract text ###############################################################
    ocr_result = reader.readtext(nplate)
    text = filter_text(region=nplate, ocr_result=ocr_result,
                        region_threshold=region_threshold)
    if len(text) == 1:
        text = text[0].upper()
        print("Text prediction using EasyOCR: ", text)
        return text
    else:
        return None
    #############################################################################################################
    
   


In [79]:
# ------------------------------------ to plot the BBox and results --------------------------------------------------------


def plot_boxes(results, frame, classes):
    """
    --> This function takes results, frame and classes
    --> results: contains labels and coordinates predicted by model on the given frame
    --> classes: contains the string labels
    """
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]

    # looping through the detections
    for i in range(n):
        row = cord[i]
        if row[4] >= 0.55:  # threshold value for detection. We are discarding everything below this value
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(
                row[2]*x_shape), int(row[3]*y_shape)  # BBOx coordniates
            coords = [x1, y1, x2, y2]
            plate_num = recognize_plate_easyocr(
                img=frame, coords=coords, reader=EASY_OCR, region_threshold=OCR_TH)
            return plate_num


In [32]:
def filter_text(region, ocr_result, region_threshold):
    """ to filter out wrong detections """
    rectangle_size = region.shape[0]*region.shape[1]

    plate = []
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))

        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
            
    return plate


In [33]:
# ---------------------------------------------- Main function -----------------------------------------------------

def license_plate_to_text(image):
    """ this function uses an image as argument, and returns the text that it extracted from the image """
    model = torch.hub.load('./yolov5', 'custom', source='local',
                           path='weights/best_submission.pt', force_reload=True)  # The repo is stored locally

    classes = model.names  # class names in string format
    # --------------- for detection on image --------------------

    results = detectx(image, model=model)  # DETECTION HAPPENING HERE

    frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    plate_num = plot_boxes(results, frame, classes=classes)
   
    return plate_num

In [34]:
import multiprocessing

def double(a):
    return a * 2


def driver_func():
    PROCESSES = 2
    with multiprocessing.Pool(PROCESSES) as pool:
        params = [(1, ), (2, ), (3, ), (4, )]
        results = [pool.apply_async(double, p) for p in params]

        for r in results:
            print('\t', r.get())


In [35]:
import multi
# driver_func()

ModuleNotFoundError: No module named 'multi'

In [36]:
multi.driver()

NameError: name 'multi' is not defined

In [78]:
# string1 = "PD32939"
# string2 = "PD22939"
# string3 = "PD32248"
# strings = [string1, string2, string3]

def filter_plate_text(strings):
    result = []
    for i in range(0, len(strings[0])):
        if strings[0][i] == strings[1][i] or strings[0][i] == strings[2][i]:
            result.append(strings[0][i])
        elif strings[1][i] == strings[2][i]:
            result.append(strings[1][i])
    # merge the list to a string
    result = "".join(result)
    return result

In [39]:
def write_to_csv_drive_in(_plate_number):   
    # Headrs for the csv files
    drive_in_colums_headers = ['Plate Number', 'Date and time']
    drive_out_colums_headers = ['Plate Number', 'Drive in time', 'Drive out time', 'Duration']

    # This will change with accual value 
    plate_number = _plate_number

    #########  get the current date and time #######
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    #############################################



    # if the header is already written, then don't write it again. If not, then write it
    if not os.path.exists('drive_in.csv'):
        with open('drive_in.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(drive_in_colums_headers)
            writer.writerow([plate_number,dt_string])
    else:
        with open('drive_in.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([plate_number,dt_string])

    if not os.path.exists('drive_out.csv'):
        with open('drive_out.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(drive_out_colums_headers)
            writer.writerow([plate_number,dt_string])
    else:
        with open('drive_out.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([plate_number,dt_string])
    return


In [40]:
def delete_row_from_csv(_plate_number, file_name):
    df = pd.read_csv(file_name+'.csv')
    df = df[df['Plate Number'] != _plate_number]
    df.to_csv('drive_in.csv', index=False)
    return

In [73]:
def write_to_csv_drive_out(_plate_number):
    """ this function takes the plate number as argument, and writes it to the drive_out.csv file """
    # This will change with accual value
    plate_number = _plate_number

    #########  get the current date and time####
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    #############################################

    # read the csv file
    df = pd.read_csv('drive_out.csv')

    # find the row with the plate number where the drive out time is empty
    assert df['Drive out time'].isnull().sum() == 1, "The car is not in the parking lot"
    row = df.loc[(df['Plate Number'] == plate_number) & (df['Drive out time'].isnull())]

    # get the index of the row
    index = row.index[0]

    # update the drive out time for the plate number
    df.at[index, 'Drive out time'] = dt_string

    # calculate the duration
    drive_in_time = df.at[index, 'Drive in time']
    drive_out_time = df.at[index, 'Drive out time']
    drive_in_time = datetime.strptime(drive_in_time, "%d/%m/%Y %H:%M:%S")
    drive_out_time = datetime.strptime(drive_out_time, "%d/%m/%Y %H:%M:%S")
    duration = drive_out_time - drive_in_time

    # update the duration
    df.at[index, 'Duration'] = duration

    # Drop the row with the plate number from the drive_in.csv file
    delete_row_from_csv(plate_number, 'drive_in')

    # save the updated csv file
    df.to_csv('drive_out.csv', index=False)

    # print the updated csv file
    print(df)
    return



In [83]:
def detect_and_log_cars(car_array):
    drive_in, images = car_array
    output = []
    # loop through the three images
    for i in range(3):
        # print "working with image number: ", i
        # call the main function
        plate_num = license_plate_to_text(images[i])
        # remve the space in the plate number
        plate_num = plate_num.replace(" ", "")
        # save the output to a list
        output.append(plate_num)
    
    detected_license_plate = filter_plate_text(output)
    print("detected_license_plate: ", detected_license_plate)

    if drive_in:
        print(f"the car with licence palte {detected_license_plate} driving in")
        write_to_csv_drive_in(detected_license_plate)
    else:
        print(f"the car with licence palte {detected_license_plate} driving out")
        write_to_csv_drive_out(detected_license_plate)

In [81]:
# image1 = cv2.imread("runs/track/exp/frame id_341.jpg")
# image2 = cv2.imread("runs/track/exp/frame id_346.jpg")
# image3 = cv2.imread("runs/track/exp/frame id_351.jpg")
# images = [image1, image2, image3]

# alex_array = []
# # add strings array to alex_array
# alex_array.append(True)
# alex_array.append(images)


In [84]:
detect_and_log_cars(alex_array)

YOLOv5 🚀 2022-12-2 Python-3.9.15 torch-1.13.0 CPU

Fusing layers... 
YOLOv5s summary: 232 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


[INFO] Detecting. . . 
Text prediciton using Pytesseract:""


YOLOv5 🚀 2022-12-2 Python-3.9.15 torch-1.13.0 CPU

Fusing layers... 
YOLOv5s summary: 232 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


Text prediction using EasyOCR:  PD 32939
[INFO] Detecting. . . 


YOLOv5 🚀 2022-12-2 Python-3.9.15 torch-1.13.0 CPU

Fusing layers... 


Text prediciton using Pytesseract:"HPD 32939}
"
Text prediction using EasyOCR:  PD 32939


YOLOv5s summary: 232 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


[INFO] Detecting. . . 
Text prediciton using Pytesseract:"IPD 32939}
"
Text prediction using EasyOCR:  PD32939
detected_license_plate:  PD32939
the car with licence palte PD32939 driving in


In [ ]:
# ALEX CODE 

def load_data(file_pth):
    data_file = glob.glob('runs/track/'+file_pth+'/tracks/*.csv')
    with open(data_file[0], 'r') as f:
        data = list(csv.reader(f))
        f.close()
        return  [[float(x) for x in row] for row in data]


def delete_data(all_ids, file_pth):
    data_file = glob.glob('runs/track/'+file_pth+'/tracks/*.csv')
    keep_rows = list()
    with open(data_file[0], 'r') as f:
        data = list(csv.reader(f))
        for row in data:
            if int(float(row[1])) not in all_ids:
                keep_rows.append(row)

    # write the rows we want to keep to the file
    with open(data_file[0], 'w') as f:
        writer = csv.writer(f)
        writer.writerows(keep_rows)

    f.close()


def sort_into_dict(data, number_of_cars_in_file):
    dict = {}
    for each_entry in data:
        frame_number = int(each_entry[0])
        car_id = int(each_entry[1])
        bounding_box_and_time = [each_entry[2:6], each_entry[6]]

        # check if id already exists in dict then append the data to the list with the frame number as key
        if car_id in dict:
            dict[car_id][frame_number] = bounding_box_and_time
        else:
            dict[car_id] = {frame_number: bounding_box_and_time}


    # remove the ids that have less than 5 entries because:
    # they might be cars that are on their way in our out of the frame or just noise
    ids_to_remove = []
    for each_id in dict: 
        if len(dict[each_id]) < number_of_cars_in_file:
            ids_to_remove.append(each_id)

    for each_id in ids_to_remove:
        del dict[each_id]

    return dict


def get_best_license_plate(car_dict,frame_width):
    car_entering = False
    sorting_list = []
    
    for frame_number in car_dict:
        # frame = each[0]
        size = car_dict[frame_number][0][-1] * car_dict[frame_number][0][-2]
        right_side = car_dict[frame_number][0][0] + car_dict[frame_number][0][2]
        left_side = car_dict[frame_number][0][0]
        
        if (left_side > 5) and (right_side < (frame_width-5)):
            sorting_list.append((frame_number, size))

    # sort the list by bounding box size
    sorting_list.sort(key=lambda x: x[1], reverse=True)

    # check if the biggest frame is the first or last frame, which means the car is entering or leaving the frame
    length_of_sort = len(sorting_list)-1
    if sorting_list[0][0] > sorting_list[length_of_sort][0] and sorting_list[0][0] > sorting_list[int(length_of_sort/2)][0]:
        car_entering = True

    # return boolean if car is entering or leaving the parking lot, and the frame numbers of the 3 biggest bounding boxes found
    return car_entering, [x[0] for x in sorting_list[:3]]


def load_image(frame_number, bounding_box_array, file_pth):
    # load image
    img = cv2.imread('runs/track/'+file_pth+'/frame id_'+str(frame_number)+'.jpg')
    # crop the image to the size of the bounding box
    img = img[int(bounding_box_array[1]):int(bounding_box_array[1]+bounding_box_array[3]), int(bounding_box_array[0]):int(bounding_box_array[0]+bounding_box_array[2])]
    return img


def get_car_images(car_dict, file_pth, frame_width):
    car_array = []
    for car_id in car_dict:
        # get the best license plate for each car
        car_entering, frames = get_best_license_plate(car_dict[car_id], frame_width)
        cropped_car_images = []
        for each_frame in frames:
            img = load_image(each_frame, car_dict[car_id][each_frame][0], file_pth)
            cropped_car_images.append(img)
        
        car_array.append([car_entering, cropped_car_images])

    return car_array


def delete_images(car_dictionary, file_pth):
    # frames with multiple cars
    # frames with no cars

    for car_id in car_dictionary:
        for frame in car_dictionary[car_id]:
            os.remove('runs/track/'+file_pth+'/frame id_'+str(frame)+'.jpg')


# TODO: if frame contains multiple cars, do not delete the image

def fetch_cars(number_of_cars_in_file, file_pth, frame_width):
    # load data from csv file
    data = load_data(file_pth)

    # sort data into dictionary
    car_dictionary = sort_into_dict(data, number_of_cars_in_file)

    # delete the loaded data from the csv file
    delete_data(list(car_dictionary.keys()), file_pth)

    # get the 3 best license plate photos for each car and the direction of the car
    car_array = get_car_images(car_dictionary, file_pth, frame_width)
    
    # delete the images containing the cars extracted
    delete_images(car_dictionary, file_pth)

    return car_array